In [1]:
%matplotlib inline
from psycopg2 import connect
import psycopg2.sql as pg
import configparser
from datetime import datetime, timedelta, date
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.cm as cm
#from matplotlib.ticker import FuncFormatter
import pandas as pd
import pandas.io.sql as pandasql
import seaborn as sns
from IPython.display import HTML
def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

In [2]:
# setting up pgsql connection
CONFIG = configparser.ConfigParser()
CONFIG.read(r'/home/jchew/local/db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [3]:
# Select bt speeds
sql = '''SELECT * FROM jchew.bt_top10_2days_spd'''
print_table(sql, con)

analysis_id,bt_id,street_name,direction,from_intersection,to_intersection,avg_tt,speed
1453262,6,Dundas,EB,Dufferin,Bathurst,387.759162,19.385229
1453284,7,Dundas,EB,Bathurst,Spadina,120.330961,20.194304
1453305,8,Dundas,EB,Spadina,University,177.632075,16.334888
1453367,11,Dundas,EB,Jarvis,Parliament,123.753561,21.817554
1453395,12,Dundas,WB,Parliament,Jarvis,140.563084,19.310902
1453445,15,Dundas,WB,University,Spadina,194.548701,14.914517
1453464,16,Dundas,WB,Spadina,Bathurst,117.133065,20.745637
1453483,17,Dundas,WB,Bathurst,Dufferin,368.212707,20.365403
1454196,53,King,EB,Spadina,University,229.272727,13.582078
1454209,54,King,EB,University,Yonge,159.576923,12.926681


In [6]:
# Finding ratio for bt speeds
sql = '''
WITH X AS (
SELECT a.analysis_id AS analysis_id_1, a.bt_id AS bt_id_1, 
	b.analysis_id AS analysis_id_2, b.bt_id AS bt_id_2, a.street_name, 
	a.direction AS eb_nb, b.direction AS wb_sb, 
	a.from_intersection AS intersection_1, a.to_intersection AS intersection_2,
	a.speed AS eb_nb_spd, b.speed AS wb_sb_spd
	FROM jchew.bt_top10_2days_spd a
JOIN jchew.bt_top10_2days_spd b
ON a.street_name = b .street_name AND a.from_intersection = b.to_intersection 
AND a.to_intersection=b.from_intersection 
WHERE a.direction IN ('EB', 'NB')
)
SELECT *,
CASE WHEN X.eb_nb_spd > X.wb_sb_spd THEN (X.eb_nb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
WHEN X.eb_nb_spd < X.wb_sb_spd THEN (X.wb_sb_spd * 1.0) / (X.eb_nb_spd + X.wb_sb_spd)
END AS "EB/WB or NB/SB Ratio",
 CASE
WHEN X.eb_nb = 'EB' THEN CASE
	WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'WB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'EB'
    END
WHEN X.eb_nb = 'NB' THEN CASE
    WHEN X.eb_nb_spd < X.wb_sb_spd THEN 'SB'
    WHEN X.eb_nb_spd > X.wb_sb_spd THEN 'NB'
    END
        END AS "Bias Towards"
FROM X
ORDER BY analysis_id_1
'''
print_table(sql, con)

analysis_id_1,bt_id_1,analysis_id_2,bt_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,eb_nb_spd,wb_sb_spd,EB/WB or NB/SB Ratio,Bias Towards
1453262,6,1453483,17,Dundas,EB,WB,Dufferin,Bathurst,19.385229,20.365403,0.512329,WB
1453284,7,1453464,16,Dundas,EB,WB,Bathurst,Spadina,20.194304,20.745637,0.506733,WB
1453305,8,1453445,15,Dundas,EB,WB,Spadina,University,16.334888,14.914517,0.522726,EB
1453367,11,1453395,12,Dundas,EB,WB,Jarvis,Parliament,21.817554,19.310902,0.530473,EB
1454196,53,1454366,62,King,EB,WB,Spadina,University,13.582078,11.842448,0.534212,EB
1454209,54,1454352,61,King,EB,WB,University,Yonge,12.926681,10.009911,0.563583,EB
1454523,70,1454683,78,Front,EB,WB,Bathurst,Spadina,22.339182,26.425829,0.541901,WB
1454549,71,1454670,77,Front,EB,WB,Spadina,University,13.175874,11.261238,0.539175,EB
1455385,117,1455256,110,University,NB,SB,Queen,Dundas,21.443348,20.945723,0.505870,NB
